In [1]:
import matplotlib.pyplot as plt 
%matplotlib inline
import metapack as mp
import pandas as pd
import geopandas as gpd
import numpy as np
from rowgenerators import RowGenerator
from sdipylib.text import make_acro
from csv import reader, writer
from os.path import exists



In [2]:
src = mp.jupyter.open_source_package()
doc = mp.jupyter.open_package()
doc

In [7]:
def clean_comm_name(s):
    import re
    return re.compile('[^a-zA-Z]').sub('',s.lower())  

from metapack.rowgenerator import PandasDataframeSource
from metapack import get_cache  
from shapely.geometry import Point
import geopandas as gpd

# First, geo join the tracts into the communities and cities. 

comm = doc.resource('community_boundaries').geoframe()
tracts = doc.resource('tracts').dataframe()

tracts['intp'] = tracts.apply(lambda r: Point(float(r.intptlon), float(r.intptlat)), axis=1)

tract_pt = gpd.GeoDataFrame(tracts, geometry = 'intp')

tract_pt.crs = comm.crs

tract_community = gpd.sjoin(comm, tract_pt, op='contains')

columns = [ 'geoid', 'type', 'name', 'name_code', 'city', 'link_code' ]

tc = tract_community.rename({'name_left':'name'}, axis=1)[columns]

# Now link everything together. 

acronyms = src.reference('acronyms')

acro_map = dict(list(acronyms)[1:])
acro_map[''] = ''


_2 = tc.set_index('geoid')

_3  = _2[_2.type == 'city'][['name','name_code']]
_4  = _2[_2.type == 'county'][['name','name_code']]
_5  = _2[_2.type == 'sd_community'][['name','name_code']]
_6  = _2[_2.type == 'county_community'][['name','name_code']]

_7 = _3.join(_4, rsuffix='_county')\
       .join(_5, rsuffix='_sdc').join(_6, rsuffix='_cnc')

_7.columns =\
[
 'city_name',
 'city_code',
 'county_name',
 'county_code',
 'community_name',
 'community_cpcode',
 'cnc_name',
 'cnc_code']

# Move the county name into the city columns, then drop it
_7['city_name'] = _7.city_name.where(~((_7.city_name.isnull()) & (_7.county_code == 'CN')),'COUNTY' )
_7['city_code'] = _7.city_code.where(~((_7.city_code.isnull()) & (_7.county_code == 'CN')),'CN' )

_7.drop(['county_name','county_code'], axis=1, inplace=True)


# Move the  county community names into the community columns
_7['community_name'] =  _7.community_name.where( _7.city_code != 'CN' ,  _7.cnc_name ).fillna('')
_7['community_cpcode'] =_7.community_cpcode.where( _7.city_code != 'CN' ,  _7.cnc_code ).fillna('0')

_7.drop(['cnc_name','cnc_code'], axis=1, inplace=True)

_7 = _7.fillna('')

_7['city_name'] = _7.city_name.apply(lambda v : v.title())
_7['community_name'] = _7.community_name.apply(lambda v : str(v).title()).fillna('')

_7['community_code'] = _7.community_name.apply(lambda v: acro_map[clean_comm_name(v)].upper() )


_7.head()


,city_name,city_code,community_name,community_cpcode,community_code
geoid,,,,,
14000US06073020207,Escondido,ES,,0,
14000US06073020211,Escondido,ES,,0,
14000US06073020209,Escondido,ES,,0,
14000US06073020206,Escondido,ES,,0,
14000US06073020210,Escondido,ES,,0,


In [10]:

_7.community_cpcode.replace('0','NA')


geoid
14000US06073020207    NA
14000US06073020211    NA
14000US06073020209    NA
14000US06073020206    NA
14000US06073020210    NA
Name: community_cpcode, dtype: object